# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Loading Dataset

In [3]:
from pandas import read_csv
# train = pd.read_csv('train_indessa.csv')
# train = pd.read_csv('train1.csv')
train = pd.read_csv('train_50k.csv')


loan_df=train.copy()
loan_df_0=train.copy()

C:\Users\smrit\AppData\Local\Temp\ipykernel_24556\4045799696.py:4: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train_50k.csv')


In [4]:
# loan_df= loan_df[:10000]

In [5]:
loan_df.shape

(49999, 45)

In [6]:
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 45 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   member_id                    49999 non-null  int64  
 1   loan_amnt                    49999 non-null  int64  
 2   funded_amnt                  49999 non-null  int64  
 3   funded_amnt_inv              49999 non-null  float64
 4   term                         49999 non-null  object 
 5   batch_enrolled               41991 non-null  object 
 6   int_rate                     49999 non-null  float64
 7   grade                        49999 non-null  object 
 8   sub_grade                    49999 non-null  object 
 9   emp_title                    47029 non-null  object 
 10  emp_length                   47418 non-null  object 
 11  home_ownership               49999 non-null  object 
 12  annual_inc                   49999 non-null  float64
 13  verification_sta

# Feature Engineering

In [7]:
!pip install category_encoders
import category_encoders as ce

In [8]:
one_hot_encoded_df = loan_df.copy()

encoder=ce.OneHotEncoder(cols= ['term', 'grade', 'home_ownership', 'verification_status', 
                                   'purpose', 'initial_list_status'], use_cat_names=True) 
                                  
one_hot_encoded_df['last_week_pay'].replace(to_replace=['th week'], value='', regex=True,inplace= True)
one_hot_encoded_df['term'] = one_hot_encoded_df['term'].map((lambda x: x.strip(' months'))).astype(str)

# Dropping unnecessary features
one_hot_encoded_df.drop(['member_id', 'batch_enrolled', 'sub_grade', 'emp_title', 'pymnt_plan', 'desc', 
                         'title', 'zip_code', 'addr_state', 'application_type', 'verification_status_joint'], axis=1, inplace= True)

# Replacing unnecessary categories with 'Others'
one_hot_encoded_df['purpose'].replace(['other', 'major_purchase', 'small_business', 'medical','car', 
                          'moving', 'vacation', 'house', 'wedding', 'renewable_energy', 
                          'educational'], 'Others', inplace=True)
one_hot_encoded_df.replace(['OTHER','NONE','ANY'],'Others',inplace=True)

# Replacing Categorical values with numeric values
one_hot_encoded_df['emp_length'].replace(to_replace=['< 1 year','1 year','2 years','3 years','4 years','5 years',  
                                          '6 years','7 years','8 years','9 years','10+ years'],
                                         value = ['1','1','2','3','4','5','6','7','8','9','10'],inplace=True)

# Numerical Features: 
# Imputing N/A values with mean, zero and Median
mean_imputation_cols= ['total_rev_hi_lim','int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'revol_bal',
         'revol_util', 'total_acc', 'total_rec_int', 'tot_cur_bal', 'total_rec_late_fee', 'recoveries',
         'collection_recovery_fee','collections_12_mths_ex_med','acc_now_delinq', 'funded_amnt_inv']
zero_imputation_cols =['delinq_2yrs','inq_last_6mths','collections_12_mths_ex_med','mths_since_last_major_derog','acc_now_delinq']
median_imputation_cols =['emp_length','annual_inc','open_acc','pub_rec','revol_util','total_acc','mths_since_last_delinq','mths_since_last_record','tot_coll_amt',
'tot_cur_bal','total_rev_hi_lim']

## Imputing N/A values with mean
one_hot_encoded_df[['total_rev_hi_lim','int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'revol_bal',
         'revol_util', 'total_acc', 'total_rec_int', 'tot_cur_bal', 'total_rec_late_fee', 'recoveries',
         'collection_recovery_fee','collections_12_mths_ex_med','acc_now_delinq', 'funded_amnt_inv']]  =   one_hot_encoded_df[['total_rev_hi_lim','int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'revol_bal',
                    'revol_util', 'total_acc', 'total_rec_int', 'tot_cur_bal', 'total_rec_late_fee', 'recoveries',
                    'collection_recovery_fee','collections_12_mths_ex_med','acc_now_delinq', 'funded_amnt_inv']].fillna(one_hot_encoded_df[
                        ['total_rev_hi_lim','int_rate', 
                            'annual_inc', 'dti', 'delinq_2yrs',
                            'inq_last_6mths', 'open_acc', 'revol_bal',
                            'revol_util', 'total_acc', 'total_rec_int', 
                            'tot_cur_bal', 'total_rec_late_fee', 'recoveries',
                            'collection_recovery_fee','collections_12_mths_ex_med',
                            'acc_now_delinq', 'funded_amnt_inv']].mean()).astype(float). astype(int)



## Imputing N/A values with zero:
for col in zero_imputation_cols:
    one_hot_encoded_df[col].fillna(0, inplace=True)
one_hot_encoded_df['last_week_pay'].replace('NA', value= 0,inplace= True)

## Imputing N/A values with median:
for col in median_imputation_cols:    
    one_hot_encoded_df[col].fillna(one_hot_encoded_df[col].median(), inplace=True)

# Replacing Categorical values with numeric values

one_hot_encoded_df['last_week_pay']=pd.to_numeric(one_hot_encoded_df.last_week_pay)
one_hot_encoded_df['term']=pd.to_numeric(one_hot_encoded_df['term'])
one_hot_encoded_df['emp_length']=pd.to_numeric(one_hot_encoded_df.emp_length)

x = encoder.fit_transform(one_hot_encoded_df)


In [9]:
x.columns.groupby(x.dtypes)

{int32: ['funded_amnt_inv', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_cur_bal', 'total_rev_hi_lim'], int64: ['loan_amnt', 'funded_amnt', 'term_36.0', 'term_60.0', 'grade_E', 'grade_B', 'grade_A', 'grade_D', 'grade_C', 'grade_F', 'grade_G', 'home_ownership_OWN', 'home_ownership_MORTGAGE', 'home_ownership_RENT', 'home_ownership_Others', 'verification_status_Source Verified', 'verification_status_Not Verified', 'verification_status_Verified', 'purpose_debt_consolidation', 'purpose_home_improvement', 'purpose_credit_card', 'purpose_Others', 'pub_rec', 'initial_list_status_f', 'initial_list_status_w', 'last_week_pay', 'loan_status'], float64: ['emp_length', 'mths_since_last_delinq', 'mths_since_last_record', 'mths_since_last_major_derog', 'tot_coll_amt']}

In [10]:
x.dropna(inplace= True)
x = x.reset_index(drop = True)
# x.isna().sum()

In [11]:
x.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term_36.0,term_60.0,int_rate,grade_E,grade_B,grade_A,grade_D,...,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status
0,14350,14350,14350,1,0,19,1,0,0,0,...,0,0,0,74.0,26,0,0.0,28699,30800,0
1,4800,4800,4800,1,0,10,0,1,0,0,...,0,0,0,0.0,9,0,0.0,9974,32900,0
2,10000,10000,10000,1,0,7,0,0,1,0,...,0,0,0,0.0,9,0,65.0,38295,34900,0
3,15000,15000,15000,1,0,19,0,0,0,1,...,0,0,0,0.0,135,0,0.0,55564,24700,0
4,16000,16000,16000,1,0,10,0,1,0,0,...,0,0,0,0.0,96,0,0.0,47159,47033,0


# Machine Learning

In [12]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kfold = KFold(n_splits=5, random_state=7, shuffle=True)

## Splitting Data into Testing and Training set

In [13]:
final_data=x.copy()
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 50 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            49999 non-null  int64  
 1   funded_amnt                          49999 non-null  int64  
 2   funded_amnt_inv                      49999 non-null  int32  
 3   term_36.0                            49999 non-null  int64  
 4   term_60.0                            49999 non-null  int64  
 5   int_rate                             49999 non-null  int32  
 6   grade_E                              49999 non-null  int64  
 7   grade_B                              49999 non-null  int64  
 8   grade_A                              49999 non-null  int64  
 9   grade_D                              49999 non-null  int64  
 10  grade_C                              49999 non-null  int64  
 11  grade_F                     

In [14]:
final_data.isna().sum()

loan_amnt                              0
funded_amnt                            0
funded_amnt_inv                        0
term_36.0                              0
term_60.0                              0
int_rate                               0
grade_E                                0
grade_B                                0
grade_A                                0
grade_D                                0
grade_C                                0
grade_F                                0
grade_G                                0
emp_length                             0
home_ownership_OWN                     0
home_ownership_MORTGAGE                0
home_ownership_RENT                    0
home_ownership_Others                  0
annual_inc                             0
verification_status_Source Verified    0
verification_status_Not Verified       0
verification_status_Verified           0
purpose_debt_consolidation             0
purpose_home_improvement               0
purpose_credit_c

In [15]:
X=final_data.drop('loan_status',axis=1)
X.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,term_36.0,term_60.0,int_rate,grade_E,grade_B,grade_A,grade_D,...,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
count,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,...,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,4.999900e+04,4.999900e+04
mean,14796.850937,14784.044681,14748.013560,0.699074,0.300926,12.672453,0.079602,0.286106,0.167163,0.155703,...,0.360487,44.184404,4.610932,0.014700,10.838437,57.299466,0.005340,195.822276,1.401636e+05,3.230018e+04
std,8401.559860,8396.864107,8407.359647,0.458665,0.458665,4.360630,0.270678,0.451944,0.373125,0.362577,...,4.266978,382.416212,60.911450,0.131014,21.907493,44.790728,0.078688,1647.042540,1.466943e+05,3.074710e+04
min,1000.000000,1000.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,8000.000000,8000.000000,8000.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,0.000000,0.000000,3.275500e+04,1.490000e+04
50%,13000.000000,13000.000000,13000.000000,1.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,48.000000,0.000000,0.000000,1.014920e+05,2.610000e+04
75%,20000.000000,20000.000000,20000.000000,1.000000,1.000000,16.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,83.000000,0.000000,0.000000,1.967825e+05,3.820000e+04
max,35000.000000,35000.000000,35000.000000,1.000000,1.000000,28.000000,1.000000,1.000000,1.000000,1.000000,...,358.000000,20006.000000,7002.000000,4.000000,150.000000,265.000000,4.000000,91996.000000,2.482331e+06,1.314900e+06


In [16]:
y = final_data['loan_status']
X=final_data.drop('loan_status',axis=1)
y = final_data['loan_status']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=101)

In [17]:
X_train.isnull().sum()


loan_amnt                              0
funded_amnt                            0
funded_amnt_inv                        0
term_36.0                              0
term_60.0                              0
int_rate                               0
grade_E                                0
grade_B                                0
grade_A                                0
grade_D                                0
grade_C                                0
grade_F                                0
grade_G                                0
emp_length                             0
home_ownership_OWN                     0
home_ownership_MORTGAGE                0
home_ownership_RENT                    0
home_ownership_Others                  0
annual_inc                             0
verification_status_Source Verified    0
verification_status_Not Verified       0
verification_status_Verified           0
purpose_debt_consolidation             0
purpose_home_improvement               0
purpose_credit_c

## Logistic Regression Model

In [18]:
from sklearn.linear_model import LogisticRegression

log_reg= LogisticRegression(max_iter=3000)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=3000)

### Evaluating Logistic Regression

In [19]:
y_pred_logreg= log_reg.predict(X_test)
print('Classification Report:\n', classification_report(y_test,y_pred_logreg))
print('\n')
print("\n Confusion matrix \n", confusion_matrix(y_test,y_pred_logreg))
print("\n Score of Logistic Regression training Model:",  log_reg.score(X_train, y_train))
print("\n Score of Logistic Regression testing Model:",  log_reg.score(X_test, y_test))
print("\n Accuracy: %.3f%%" % ((log_reg.score(X_test, y_test))*100.0))

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.97      0.87     11474
           1       0.66      0.21      0.31      3526

    accuracy                           0.79     15000
   macro avg       0.73      0.59      0.59     15000
weighted avg       0.77      0.79      0.74     15000




 Confusion matrix 
 [[11100   374]
 [ 2798   728]]

 Score of Random Forest training Model: 0.7838223949255693

 Score of Random Forest testing Model: 0.7885333333333333

 Accuracy: 78.853%


## Decision Tree Classifier

In [20]:
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier()
dtree.fit(X_train,y_train)

DecisionTreeClassifier()

### Evaluating Decision Tree

In [21]:
y_pred_dtree=dtree.predict(X_test)
print('Classification Report:\n', classification_report(y_test,y_pred_dtree))
print('\n')
print("\n Confusion matrix \n", confusion_matrix(y_test,y_pred_dtree))
print("\n Score of Decision Tree training Model:",  dtree.score(X_train, y_train))
print("\n Score of Decision Tree testing Model:",  dtree.score(X_test, y_test))
print("\n Accuracy: %.3f%%" % ((dtree.score(X_test, y_test))*100.0))

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.82      0.83     11474
           1       0.47      0.52      0.49      3526

    accuracy                           0.75     15000
   macro avg       0.66      0.67      0.66     15000
weighted avg       0.76      0.75      0.75     15000




 Confusion matrix 
 [[9418 2056]
 [1707 1819]]

 Score of Random Forest training Model: 1.0

 Score of Random Forest testing Model: 0.7491333333333333

 Accuracy: 74.913%


## Random Forest Classifier

In [22]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=600)
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=600)

### Evaluating Random Forest Classifier:

In [23]:
y_pred_rfc=rfc.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print('Classification Report:\n', classification_report(y_test,y_pred_rfc))
print('\n')
print("\n Confusion matrix \n", confusion_matrix(y_test,y_pred_rfc))
print("\n Score of Random Forest training Model:",  rfc.score(X_train, y_train))
print("\n Score of Random Forest testing Model:",  rfc.score(X_test, y_test))
print("\n Accuracy: %.3f%%" % ((rfc.score(X_test, y_test))*100.0))

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.99      0.90     11474
           1       0.92      0.32      0.47      3526

    accuracy                           0.83     15000
   macro avg       0.87      0.66      0.69     15000
weighted avg       0.85      0.83      0.80     15000




 Confusion matrix 
 [[11381    93]
 [ 2403  1123]]

 Score of Random Forest training Model: 1.0

 Score of Random Forest testing Model: 0.8336

 Accuracy: 83.360%


## K- Nearest Neighbor Classification

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Trying Out different n-Neighbors

#### Nearest Neighbor = 1

In [25]:
knn1 = KNeighborsClassifier(n_neighbors = 1)
knn1.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [26]:
y_pred_knn1=knn1.predict(X_test)

#### Nearest Neighbor = 5

In [27]:
knn5 = KNeighborsClassifier(n_neighbors = 5)
knn5.fit(X_train, y_train)
y_pred_knn5=knn5.predict(X_test)


#### Nearest Neighbor = 7

In [28]:
knn7 = KNeighborsClassifier(n_neighbors = 7)
knn7.fit(X_train, y_train)
y_pred_knn7=knn7.predict(X_test)


#### Nearest Neighbor = 10

In [29]:
knn10 = KNeighborsClassifier(n_neighbors = 10)
knn10.fit(X_train, y_train)
y_pred_knn10=knn10.predict(X_test)


### Evaluation Metrics

In [30]:
print('KNN =1')
print('Classification Report:\n', classification_report(y_test,y_pred_knn1))
print('\n')
print("\n Confusion matrix \n", confusion_matrix(y_test,y_pred_knn1))
print("\n Score of Random Forest training Model:",  knn1.score(X_train, y_train))
print("\n Score of Random Forest testing Model:",  knn1.score(X_test, y_test))
print("\n Accuracy: %.3f%%" % ((knn1.score(X_test, y_test))*100.0))

KNN =1
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.80      0.80     11474
           1       0.38      0.40      0.39      3526

    accuracy                           0.70     15000
   macro avg       0.59      0.60      0.60     15000
weighted avg       0.71      0.70      0.71     15000




 Confusion matrix 
 [[9157 2317]
 [2130 1396]]

 Score of Random Forest training Model: 1.0

 Score of Random Forest testing Model: 0.7035333333333333

 Accuracy: 70.353%


In [31]:
print('KNN = 5')
print('Classification Report:\n', classification_report(y_test,y_pred_knn5))
print('\n')
print("\n Confusion matrix \n", confusion_matrix(y_test,y_pred_knn5))
print("\n Score of Random Forest training Model:",  knn5.score(X_train, y_train))
print("\n Score of Random Forest testing Model:",  knn5.score(X_test, y_test))
print("\n Accuracy: %.3f%%" % ((knn5.score(X_test, y_test))*100.0))

KNN = 5
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.90      0.85     11474
           1       0.50      0.32      0.39      3526

    accuracy                           0.76     15000
   macro avg       0.65      0.61      0.62     15000
weighted avg       0.74      0.76      0.74     15000




 Confusion matrix 
 [[10329  1145]
 [ 2397  1129]]

 Score of Random Forest training Model: 0.8297951370039144

 Score of Random Forest testing Model: 0.7638666666666667

 Accuracy: 76.387%


In [32]:
print('KNN = 7')
print('Classification Report:\n', classification_report(y_test,y_pred_knn7))
print('\n')
print("\n Confusion matrix \n", confusion_matrix(y_test,y_pred_knn7))
print("\n Score of Random Forest training Model:",  knn7.score(X_train, y_train))
print("\n Score of Random Forest testing Model:",  knn7.score(X_test, y_test))
print("\n Accuracy: %.3f%%" % ((knn7.score(X_test, y_test))*100.0))

KNN = 7
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.92      0.86     11474
           1       0.54      0.30      0.38      3526

    accuracy                           0.77     15000
   macro avg       0.67      0.61      0.62     15000
weighted avg       0.75      0.77      0.75     15000




 Confusion matrix 
 [[10566   908]
 [ 2477  1049]]

 Score of Random Forest training Model: 0.8192519786279608

 Score of Random Forest testing Model: 0.7743333333333333

 Accuracy: 77.433%


In [33]:
print('KNN = 10')
print('Classification Report:\n', classification_report(y_test,y_pred_knn10))
print('\n')
print("\n Confusion matrix \n", confusion_matrix(y_test,y_pred_knn10))
print("\n Score of Random Forest training Model:",  knn10.score(X_train, y_train))
print("\n Score of Random Forest testing Model:",  knn10.score(X_test, y_test))
print("\n Accuracy: %.3f%%" % ((knn1.score(X_test, y_test))*100.0))

KNN = 10
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.96      0.87     11474
           1       0.62      0.23      0.34      3526

    accuracy                           0.79     15000
   macro avg       0.71      0.59      0.60     15000
weighted avg       0.76      0.79      0.75     15000




 Confusion matrix 
 [[10971   503]
 [ 2710   816]]

 Score of Random Forest training Model: 0.8077087916797623

 Score of Random Forest testing Model: 0.7858

 Accuracy: 70.353%


## Gradient Boosting

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [35]:
# 1st method
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01],
    "min_samples_split": [0.1],
    "min_samples_leaf": [0.1],
    "max_depth":[3],
    "max_features":["log2"],
    "criterion": ["friedman_mse"],
    "subsample":[0.5],
    "n_estimators":[10]
    }
gb_clfb_1 = GridSearchCV(GradientBoostingClassifier(), parameters, cv=5, n_jobs=-1,verbose=1)
gb_clfb_1.fit(X_train, y_train)
print(gb_clfb_1.score(X_train, y_train))

print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",gb_clfb_1.best_estimator_)
print("\n The best score across ALL searched params:\n",gb_clfb_1.best_score_)
print("\n The best parameters across ALL searched params:\n",gb_clfb_1.best_params_)
print("\n Accuracy Score:\n", accuracy_score(gb_clfb_1.predict(X_test), y_test)*100)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
0.7611931769479128
 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingClassifier(learning_rate=0.01, max_features='log2',
                           min_samples_leaf=0.1, min_samples_split=0.1,
                           n_estimators=10, subsample=0.5)

 The best score across ALL searched params:
 0.7611931786173535

 The best parameters across ALL searched params:
 {'criterion': 'friedman_mse', 'learning_rate': 0.01, 'loss': 'deviance', 'max_depth': 3, 'max_features': 'log2', 'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'n_estimators': 10, 'subsample': 0.5}

 Accuracy Score:
 76.49333333333334


In [36]:
# 2nd method
parameters = {
    "learning_rate": [0.01, 0.1, 0.2],
    "min_samples_split": [0.1],
    "min_samples_leaf": [0.1],
    "max_depth":[5, 10, 15, 20],
    "max_features":["log2","sqrt"],
    "subsample":[0.7, 0.8, 0.9],
    "n_estimators":[100,200,300]
    }

# should be right 

gb_clfb = GridSearchCV(GradientBoostingClassifier(), parameters, cv=5, n_jobs=-1,verbose=4)
gb_clfb.fit(X_train, y_train)
print(gb_clfb.score(X_train, y_train))
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",gb_clfb.best_estimator_)
print("\n The best score across ALL searched params:\n",gb_clfb.best_score_)
print("\n The best parameters across ALL searched params:\n",gb_clfb.best_params_)
print("\n Accuracy Score:\n", accuracy_score(gb_clfb.predict(X_test), y_test)*100)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
0.8178519386268179
 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingClassifier(learning_rate=0.2, max_depth=5, max_features='sqrt',
                           min_samples_leaf=0.1, min_samples_split=0.1,
                           n_estimators=300, subsample=0.9)

 The best score across ALL searched params:
 0.8114516726879349

 The best parameters across ALL searched params:
 {'learning_rate': 0.2, 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'n_estimators': 300, 'subsample': 0.9}

 Accuracy Score:
 81.58


In [37]:
# Predictions:
y_pred_gb=gb_clfb.predict(X_test)

In [38]:
import pickle
model = gb_clfb.best_estimator_

with open('Gradient Boost tuned.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(model, file)



### Evaluating Gradient Boosting Classifier:

In [39]:
# For visualisation:
pd.crosstab(y_test, y_pred_gb, rownames=['Actual'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
Actual,,,
0,11061,413,11474
1,2350,1176,3526
All,13411,1589,15000


In [40]:
print('Classification Report:\n', classification_report(y_test,y_pred_gb))
print('\n')
print("\n Confusion matrix \n", confusion_matrix(y_test,y_pred_gb))
print("\n Score of Random Forest training Model:",  gb_clfb.score(X_train, y_train))
print("\n Score of Random Forest testing Model:",  gb_clfb.score(X_test, y_test))
print("\n Accuracy: %.3f%%" % ((gb_clfb.score(X_test, y_test))*100.0))

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.96      0.89     11474
           1       0.74      0.33      0.46      3526

    accuracy                           0.82     15000
   macro avg       0.78      0.65      0.67     15000
weighted avg       0.80      0.82      0.79     15000




 Confusion matrix 
 [[11061   413]
 [ 2350  1176]]

 Score of Random Forest training Model: 0.8178519386268179

 Score of Random Forest testing Model: 0.8158

 Accuracy: 81.580%


# Comparing Models



In [41]:
# Import required libraries for performance metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate

# dictionary with performance metrics
scoring = {'accuracy':make_scorer(accuracy_score), 
           'precision':make_scorer(precision_score),
           'recall':make_scorer(recall_score), 
           'f1_score':make_scorer(f1_score)}


# Models evaluation function
def models_evaluation(X, y, folds):
    
    # Performing cross-validation to each classifier
    log_cv = cross_validate(log_reg, X, y, cv=folds, scoring=scoring)
    dtree_cv = cross_validate(dtree, X, y, cv=folds, scoring=scoring)
    rfc_cv = cross_validate(rfc, X, y, cv=folds, scoring=scoring)
    knn1_cv = cross_validate(knn1, X, y, cv=folds, scoring=scoring)
    knn5_cv = cross_validate(knn5, X, y, cv=folds, scoring=scoring)
    knn7_cv = cross_validate(knn7, X, y, cv=folds, scoring=scoring)
    knn10_cv = cross_validate(knn10, X, y, cv=folds, scoring=scoring)
    gb_cv = cross_validate(gb_clfb, X, y, cv=folds, scoring=scoring)

    # Creating a data frame with the model performance metrics:
    models_scores_table = pd.DataFrame({'Logistic Regression':[log_cv['test_accuracy'].mean(),
                                                               log_cv['test_precision'].mean(),
                                                               log_cv['test_recall'].mean(),
                                                               log_cv['test_f1_score'].mean()],
                                       
                                      'Decision Tree':[dtree_cv['test_accuracy'].mean(),
                                                                   dtree_cv['test_precision'].mean(),
                                                                   dtree_cv['test_recall'].mean(),
                                                                   dtree_cv['test_f1_score'].mean()],
                                       
                                      'Random Forest Classifier':[rfc_cv['test_accuracy'].mean(),
                                                       rfc_cv['test_precision'].mean(),
                                                       rfc_cv['test_recall'].mean(),
                                                       rfc_cv['test_f1_score'].mean()],
                                       
                                      'K-Nearest Neighbor (1)':[knn1_cv['test_accuracy'].mean(),
                                                       knn1_cv['test_precision'].mean(),
                                                       knn1_cv['test_recall'].mean(),
                                                       knn1_cv['test_f1_score'].mean()],
                                        
                                      'K-Nearest Neighbor (nearest neighbors= 5)':[knn5_cv['test_accuracy'].mean(),
                                                       knn5_cv['test_precision'].mean(),
                                                       knn5_cv['test_recall'].mean(),
                                                       knn5_cv['test_f1_score'].mean()],
                                       
                                      'K-Nearest Neighbor (7)':[knn7_cv['test_accuracy'].mean(),
                                                       knn7_cv['test_precision'].mean(),
                                                       knn7_cv['test_recall'].mean(),
                                                       knn7_cv['test_f1_score'].mean()],
                                       
                                      'K-Nearest Neighbor (10)':[knn10_cv['test_accuracy'].mean(),
                                                       knn10_cv['test_precision'].mean(),
                                                       knn10_cv['test_recall'].mean(),
                                                       knn10_cv['test_f1_score'].mean()],
                                       
                                       
                                      'Gradient Boosting':[gb_cv['test_accuracy'].mean(),
                                                              gb_cv['test_precision'].mean(),
                                                              gb_cv['test_recall'].mean(),
                                                              gb_cv['test_f1_score'].mean()]},
                                      
                                      index=['Accuracy', 'Precision', 'Recall', 'F1 Score'])
    
    # Adding 'Best Score' column
    models_scores_table['Best Score'] = models_scores_table.idxmax(axis=1)
    
    # Returning models performance metrics in the form of data frame
    return(models_scores_table)
  
# Calling models_evaluation function with 5 folds
results = models_evaluation(X_test, y_test, 5)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Fitting 5 folds for each of 216 candidates, totalling 1080 fits


In [42]:
results.T

,Accuracy,Precision,Recall,F1 Score
Logistic Regression,0.7956,0.635068,0.308562,0.415213
Decision Tree,0.736,0.443001,0.478726,0.460136
Random Forest Classifier,0.831933,0.921202,0.311679,0.46564
K-Nearest Neighbor (1),0.705267,0.373983,0.376627,0.375252
K-Nearest Neighbor (nearest neighbors= 5),0.7662,0.505126,0.299775,0.376056
K-Nearest Neighbor (7),0.776267,0.547125,0.284176,0.373958
K-Nearest Neighbor (10),0.7862,0.63625,0.211002,0.316891
Gradient Boosting,0.8112,0.736407,0.306857,0.433159
Best Score,Random Forest Classifier,Random Forest Classifier,Decision Tree,Random Forest Classifier


In [44]:
results

,Logistic Regression,Decision Tree,Random Forest Classifier,K-Nearest Neighbor (1),K-Nearest Neighbor (nearest neighbors= 5),K-Nearest Neighbor (7),K-Nearest Neighbor (10),Gradient Boosting,Best Score
Accuracy,0.795600,0.736000,0.831933,0.705267,0.766200,0.776267,0.786200,0.811200,Random Forest Classifier
Precision,0.635068,0.443001,0.921202,0.373983,0.505126,0.547125,0.636250,0.736407,Random Forest Classifier
Recall,0.308562,0.478726,0.311679,0.376627,0.299775,0.284176,0.211002,0.306857,Decision Tree
F1 Score,0.415213,0.460136,0.465640,0.375252,0.376056,0.373958,0.316891,0.433159,Random Forest Classifier
